**Driving Question :** What is the difference between the Althoff et al (2017) calculation of step counts with one that requires 10-hours of the day with steps for a small number of users?

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
df = pd.read_csv("/content/User1.csv")

In [3]:
df

,Start,Finish,Steps (count)
0,07-Dec-2014 09:00,07-Dec-2014 10:00,941.0
1,07-Dec-2014 10:00,07-Dec-2014 11:00,408.0
2,07-Dec-2014 11:00,07-Dec-2014 12:00,157.0
3,07-Dec-2014 12:00,07-Dec-2014 13:00,1017.0
4,07-Dec-2014 13:00,07-Dec-2014 14:00,0.0
...,...,...,...
42071,25-Sep-2019 07:00,25-Sep-2019 08:00,0.0
42072,25-Sep-2019 08:00,25-Sep-2019 09:00,0.0
42073,25-Sep-2019 09:00,25-Sep-2019 10:00,0.0
42074,25-Sep-2019 10:00,25-Sep-2019 11:00,0.0


In [4]:
df["Date"] = [s.split(" ")[0] for s in df["Start"]]
df["Hour"] = [int((s.split(":")[0])[-2:]) for s in df["Start"]]
df 

,Start,Finish,Steps (count),Date,Hour
0,07-Dec-2014 09:00,07-Dec-2014 10:00,941.0,07-Dec-2014,9
1,07-Dec-2014 10:00,07-Dec-2014 11:00,408.0,07-Dec-2014,10
2,07-Dec-2014 11:00,07-Dec-2014 12:00,157.0,07-Dec-2014,11
3,07-Dec-2014 12:00,07-Dec-2014 13:00,1017.0,07-Dec-2014,12
4,07-Dec-2014 13:00,07-Dec-2014 14:00,0.0,07-Dec-2014,13
...,...,...,...,...,...
42071,25-Sep-2019 07:00,25-Sep-2019 08:00,0.0,25-Sep-2019,7
42072,25-Sep-2019 08:00,25-Sep-2019 09:00,0.0,25-Sep-2019,8
42073,25-Sep-2019 09:00,25-Sep-2019 10:00,0.0,25-Sep-2019,9
42074,25-Sep-2019 10:00,25-Sep-2019 11:00,0.0,25-Sep-2019,10


In [5]:
df1 = df.value_counts("Date") 
df1 

# issues of daylight savings, and not starting from the start of a day (first and last entry of dataset)

Date
03-Apr-2016    25
02-Apr-2017    25
01-Apr-2018    25
05-Apr-2015    25
07-Apr-2019    25
               ..
01-Oct-2017    23
02-Oct-2016    23
04-Oct-2015    23
07-Dec-2014    15
25-Sep-2019    12
Length: 1754, dtype: int64

In [6]:
#Valid day is a day greater than 500 steps total
df1 = df['Steps (count)'].groupby(df['Date']).sum()
print(df1)

Date
01-Apr-2015     7828.000000
01-Apr-2016     6831.350375
01-Apr-2017     4122.000000
01-Apr-2018    11518.000000
01-Apr-2019     5484.000000
                   ...     
31-May-2019     6424.000000
31-Oct-2015     7534.000000
31-Oct-2016      453.000000
31-Oct-2017     2198.000000
31-Oct-2018     6444.139901
Name: Steps (count), Length: 1754, dtype: float64


In [7]:
#Number of days with steps greater than 1533, original = 1754
sum(df1 > 500)

1533

In [8]:
#Valid day is if there is data within 3 time periods, 3am-11am,11am-3pm,3pm-3am
bins = [-1,2,10,14,25,np.inf]
names = ['3','1','2', '3', '3']
df2 = df
df2['3timeblock'] = pd.cut(df['Hour'],bins, labels = names, ordered=False)
df2

,Start,Finish,Steps (count),Date,Hour,3timeblock
0,07-Dec-2014 09:00,07-Dec-2014 10:00,941.0,07-Dec-2014,9,1
1,07-Dec-2014 10:00,07-Dec-2014 11:00,408.0,07-Dec-2014,10,1
2,07-Dec-2014 11:00,07-Dec-2014 12:00,157.0,07-Dec-2014,11,2
3,07-Dec-2014 12:00,07-Dec-2014 13:00,1017.0,07-Dec-2014,12,2
4,07-Dec-2014 13:00,07-Dec-2014 14:00,0.0,07-Dec-2014,13,2
...,...,...,...,...,...,...
42071,25-Sep-2019 07:00,25-Sep-2019 08:00,0.0,25-Sep-2019,7,1
42072,25-Sep-2019 08:00,25-Sep-2019 09:00,0.0,25-Sep-2019,8,1
42073,25-Sep-2019 09:00,25-Sep-2019 10:00,0.0,25-Sep-2019,9,1
42074,25-Sep-2019 10:00,25-Sep-2019 11:00,0.0,25-Sep-2019,10,1


In [9]:
df2 = df2.groupby(['Date','3timeblock'], as_index=False)['Steps (count)'].sum()
print(df2)
df2['Activity'] = df2['Steps (count)'] > 0
print(df2)

             Date 3timeblock  Steps (count)
0     01-Apr-2015          1    1721.000000
1     01-Apr-2015          2    4105.000000
2     01-Apr-2015          3    2002.000000
3     01-Apr-2016          1    3671.836598
4     01-Apr-2016          2    1608.797745
...           ...        ...            ...
5257  31-Oct-2017          2       0.000000
5258  31-Oct-2017          3    1425.000000
5259  31-Oct-2018          1    2148.046634
5260  31-Oct-2018          2    1074.023317
5261  31-Oct-2018          3    3222.069951

[5262 rows x 3 columns]
             Date 3timeblock  Steps (count)  Activity
0     01-Apr-2015          1    1721.000000      True
1     01-Apr-2015          2    4105.000000      True
2     01-Apr-2015          3    2002.000000      True
3     01-Apr-2016          1    3671.836598      True
4     01-Apr-2016          2    1608.797745      True
...           ...        ...            ...       ...
5257  31-Oct-2017          2       0.000000     False
5258  31-Oct-20

In [10]:
df2 = df2['Activity'].groupby(df2['Date']).sum()
sum(df2==3)

# Original is 1754


985

In [10]:
#10 hours continous 
df2['Activity'] = df2['Steps (count)'] > 0
